In [ ]:
{
  "nbformat": 4,
  "nbformat_minor": 5,
  "metadata": {
    "colab": {
      "name": "Інтелектуальна система аналізу та прогнозування продажів.ipynb",
      "provenance": [],
      "collapsed_sections": []
    },
    "kernelspec": {
      "name": "python3",
      "display_name": "Python 3"
    },
    "language_info": {
      "name": "python",
      "version": "3.x"
    }
  },
  "cells": [
    {
      "cell_type": "markdown",
      "metadata": {},
      "source": [
        "# Інтелектуальна система аналізу та прогнозування продажів\n",
        "## на основі неструктурованих Excel-звітів\n",
        "\n",
        "**Дипломна робота**\n",
        "\n",
        "- Студент: _ПІБ_\n",
        "- Група: _Група_\n",
        "- Керівник: _ПІБ_\n",
        "- Рік: 2026\n",
        "\n",
        "---\n",
        "\n",
        "### Анотація\n",
        "У даній роботі розроблено підхід до автоматизованого аналізу та прогнозування продажів на основі неструктурованих Excel-звітів.\n",
        "Рішення включає етапи: зчитування та уніфікація даних, очистка, EDA (у тому числі boxplot), кластеризація об’єктів (клієнтів/товарів) та побудова моделей машинного навчання для прогнозування.\n"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {},
      "source": [
        "## 1. Вступ\n",
        "\n",
        "### 1.1 Актуальність\n",
        "У багатьох організаціях звіти з продажів зберігаються у вигляді Excel-файлів без сталої структури: різні назви колонок, додаткові заголовки, змішані формати дат, пропуски та дублікати. Це ускладнює швидкий аналіз і унеможливлює відтворюваний прогноз.\n",
        "\n",
        "### 1.2 Мета та завдання\n",
        "**Мета:** створення інтелектуальної системи аналізу та прогнозування продажів на основі неструктурованих Excel-звітів.\n",
        "\n",
        "**Завдання:**\n",
        "1) імпорт та уніфікація даних з Excel;\n",
        "2) очистка та підготовка;\n",
        "3) EDA (статистики, розподіли, boxplot, кореляції);\n",
        "4) формування ознак (feature engineering);\n",
        "5) кластеризація (segmentation);\n",
        "6) побудова моделей прогнозування (baseline + покращення);\n",
        "7) оцінка якості та інтерпретація результатів.\n"
      ]
    },
    {
      "cell_type": "code",
      "metadata": {},
      "execution_count": null,
      "outputs": [],
      "source": [
        "# 2. Імпорт бібліотек та налаштування\n",
        "\n",
        "import os\n",
        "import re\n",
        "import warnings\n",
        "warnings.filterwarnings('ignore')\n",
        "\n",
        "import numpy as np\n",
        "import pandas as pd\n",
        "\n",
        "import matplotlib.pyplot as plt\n",
        "\n",
        "from sklearn.model_selection import train_test_split\n",
        "from sklearn.preprocessing import StandardScaler\n",
        "from sklearn.cluster import KMeans\n",
        "from sklearn.metrics import silhouette_score\n",
        "from sklearn.metrics import mean_absolute_error, mean_squared_error\n",
        "\n",
        "from sklearn.linear_model import LinearRegression\n",
        "from sklearn.ensemble import RandomForestRegressor\n",
        "\n",
        "pd.set_option('display.max_columns', 200)\n",
        "pd.set_option('display.width', 120)\n",
        "\n",
        "print('OK')\n"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {},
      "source": [
        "## 3. Завантаження даних\n",
        "\n",
        "Нижче реалізовано два сценарії:\n",
        "1) **Основний**: дані читаються з Excel у Google Drive (Colab).\n",
        "2) **Демонстраційний**: якщо файл не знайдено, генерується невеликий синтетичний набір даних для перевірки пайплайна.\n"
      ]
    },
    {
      "cell_type": "code",
      "metadata": {},
      "execution_count": null,
      "outputs": [],
      "source": [
        "# Якщо працюєш у Colab — розкоментуй блок нижче\n",
        "# from google.colab import drive\n",
        "# drive.mount('/content/drive')\n",
        "\n",
        "# Задай шлях до Excel-файлу (приклад для Colab/Drive):\n",
        "# EXCEL_PATH = '/content/drive/MyDrive/diploma/data/sales.xlsx'\n",
        "\n",
        "# Для PyCharm/локально — наприклад:\n",
        "EXCEL_PATH = './data/sales.xlsx'\n",
        "\n",
        "def _normalize_col_name(s: str) -> str:\n",
        "    s = str(s).strip().lower()\n",
        "    s = re.sub(r'\\s+', '_', s)\n",
        "    s = re.sub(r'[^a-z0-9_а-яіїєґ]+', '', s)\n",
        "    return s\n",
        "\n",
        "def load_sales_excel(path: str) -> pd.DataFrame:\n",
        "    # Підтримка багатолистових Excel: зчитуємо перший лист за замовчуванням.\n",
        "    df = pd.read_excel(path)\n",
        "    df.columns = [_normalize_col_name(c) for c in df.columns]\n",
        "    return df\n",
        "\n",
        "def make_demo_dataset(n=2000, seed=42) -> pd.DataFrame:\n",
        "    rng = np.random.default_rng(seed)\n",
        "    dates = pd.date_range('2024-01-01', periods=365, freq='D')\n",
        "    df = pd.DataFrame({\n",
        "        'date': rng.choice(dates, size=n),\n",
        "        'product_id': rng.integers(1, 120, size=n),\n",
        "        'store_id': rng.integers(1, 12, size=n),\n",
        "        'price': np.round(rng.normal(250, 80, size=n).clip(20, 900), 2),\n",
        "        'quantity': rng.integers(1, 8, size=n),\n",
        "    })\n",
        "    # Синтетична формула продажів + шум\n",
        "    season = 1.0 + 0.15*np.sin(2*np.pi*df['date'].dt.dayofyear/365)\n",
        "    df['sales'] = np.round(df['price'] * df['quantity'] * season * rng.normal(1.0, 0.15, size=n), 2)\n",
        "    return df.sort_values('date').reset_index(drop=True)\n",
        "\n",
        "if os.path.exists(EXCEL_PATH):\n",
        "    df = load_sales_excel(EXCEL_PATH)\n",
        "    print('Завантажено Excel:', EXCEL_PATH)\n",
        "else:\n",
        "    df = make_demo_dataset()\n",
        "    print('Файл не знайдено. Використано демонстраційний датасет.')\n",
        "\n",
        "df.head(10)\n"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {},
      "source": [
        "## 4. Первинний аналіз (Data Understanding)\n",
        "\n",
        "У цьому розділі:\n",
        "- перевіряємо типи та пропуски;\n",
        "- оцінюємо базові статистики;\n",
        "- фіксуємо припущення щодо структури даних.\n"
      ]
    },
    {
      "cell_type": "code",
      "metadata": {},
      "execution_count": null,
      "outputs": [],
      "source": [
        "df.info()\n"
      ]
    },
    {
      "cell_type": "code",
      "metadata": {},
      "execution_count": null,
      "outputs": [],
      "source": [
        "df.isna().sum().sort_values(ascending=False).head(20)\n"
      ]
    },
    {
      "cell_type": "code",
      "metadata": {},
      "execution_count": null,
      "outputs": [],
      "source": [
        "df.describe(include='all').T.head(30)\n"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {},
      "source": [
        "## 5. Очистка та підготовка даних\n",
        "\n",
        "### 5.1 Уніфікація типів\n",
        "- дата → `datetime`\n",
        "- числові поля → `float/int`\n",
        "\n",
        "### 5.2 Обробка пропусків та аномалій\n",
        "Стратегія залежить від джерела даних. Для шаблону використано помірні дефолти.\n"
      ]
    },
    {
      "cell_type": "code",
      "metadata": {},
      "execution_count": null,
      "outputs": [],
      "source": [
        "# Гарантуємо наявність потрібних колонок для пайплайна.\n",
        "# Якщо у твоєму Excel інші назви — просто адаптуй цей словник.\n",
        "RENAME_MAP = {\n",
        "    'дата': 'date',\n",
        "    'data': 'date',\n",
        "    'товар': 'product_id',\n",
        "    'product': 'product_id',\n",
        "    'ціна': 'price',\n",
        "    'price_uah': 'price',\n",
        "    'кількість': 'quantity',\n",
        "    'qty': 'quantity',\n",
        "    'продажі': 'sales',\n",
        "    'revenue': 'sales',\n",
        "}\n",
        "\n",
        "df = df.rename(columns={c: RENAME_MAP.get(c, c) for c in df.columns})\n",
        "\n",
        "if 'date' in df.columns:\n",
        "    df['date'] = pd.to_datetime(df['date'], errors='coerce')\n",
        "\n",
        "for col in ['price', 'quantity', 'sales']:\n",
        "    if col in df.columns:\n",
        "        df[col] = pd.to_numeric(df[col], errors='coerce')\n",
        "\n",
        "# Мінімальна очистка: прибираємо рядки без критичних полів\n",
        "critical = [c for c in ['date', 'price', 'quantity', 'sales'] if c in df.columns]\n",
        "df_clean = df.dropna(subset=critical).copy()\n",
        "\n",
        "# Прибираємо очевидні невалідні значення\n",
        "if 'price' in df_clean.columns:\n",
        "    df_clean = df_clean[df_clean['price'] >= 0]\n",
        "if 'quantity' in df_clean.columns:\n",
        "    df_clean = df_clean[df_clean['quantity'] >= 0]\n",
        "if 'sales' in df_clean.columns:\n",
        "    df_clean = df_clean[df_clean['sales'] >= 0]\n",
        "\n",
        "df_clean = df_clean.sort_values('date').reset_index(drop=True)\n",
        "df_clean.head(10)\n"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {},
      "source": [
        "## 6. EDA (Exploratory Data Analysis)\n",
        "\n",
        "У цьому розділі:\n",
        "- розподіли продажів;\n",
        "- boxplot (\"ящики з вусами\") для пошуку викидів;\n",
        "- кореляції між ознаками.\n"
      ]
    },
    {
      "cell_type": "code",
      "metadata": {},
      "execution_count": null,
      "outputs": [],
      "source": [
        "numeric_cols = [c for c in ['sales', 'price', 'quantity'] if c in df_clean.columns]\n",
        "df_clean[numeric_cols].describe().T\n"
      ]
    },
    {
      "cell_type": "code",
      "metadata": {},
      "execution_count": null,
      "outputs": [],
      "source": [
        "# Розподіл продажів\n",
        "if 'sales' in df_clean.columns:\n",
        "    plt.figure(figsize=(8, 4))\n",
        "    plt.hist(df_clean['sales'], bins=40)\n",
        "    plt.title('Розподіл продажів (sales)')\n",
        "    plt.xlabel('sales')\n",
        "    plt.ylabel('К-сть')\n",
        "    plt.show()\n"
      ]
    },
    {
      "cell_type": "code",
      "metadata": {},
      "execution_count": null,
      "outputs": [],
      "source": [
        "# Boxplot (ящики з вусами)\n",
        "if 'sales' in df_clean.columns:\n",
        "    plt.figure(figsize=(8, 2.5))\n",
        "    plt.boxplot(df_clean['sales'].dropna(), vert=False)\n",
        "    plt.title('Boxplot для sales')\n",
        "    plt.xlabel('sales')\n",
        "    plt.show()\n"
      ]
    },
    {
      "cell_type": "code",
      "metadata": {},
      "execution_count": null,
      "outputs": [],
      "source": [
        "# Кореляції\n",
        "corr_cols = [c for c in ['sales', 'price', 'quantity'] if c in df_clean.columns]\n",
        "if len(corr_cols) >= 2:\n",
        "    corr = df_clean[corr_cols].corr(numeric_only=True)\n",
        "    print(corr)\n",
        "    plt.figure(figsize=(4.5, 3.5))\n",
        "    plt.imshow(corr, aspect='auto')\n",
        "    plt.xticks(range(len(corr_cols)), corr_cols, rotation=45)\n",
        "    plt.yticks(range(len(corr_cols)), corr_cols)\n",
        "    plt.title('Кореляційна матриця (простий вигляд)')\n",
        "    plt.colorbar()\n",
        "    plt.tight_layout()\n",
        "    plt.show()\n"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {},
      "source": [
        "## 7. Формування ознак (Feature Engineering)\n",
        "\n",
        "Приклади ознак:\n",
        "- календарні: місяць, день тижня;\n",
        "- агрегації: продажі по товару/магазину;\n",
        "- лаги та rolling-статистики (за потреби, якщо робиш часовий прогноз).\n"
      ]
    },
    {
      "cell_type": "code",
      "metadata": {},
      "execution_count": null,
      "outputs": [],
      "source": [
        "df_fe = df_clean.copy()\n",
        "\n",
        "if 'date' in df_fe.columns:\n",
        "    df_fe['month'] = df_fe['date'].dt.month\n",
        "    df_fe['dow'] = df_fe['date'].dt.dayofweek\n",
        "\n",
        "df_fe.head(10)\n"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {},
      "source": [
        "## 8. Кластеризація (Segmentation)\n",
        "\n",
        "Далі показано приклад кластеризації **товарів** за агрегованими характеристиками.\n",
        "За потреби можна аналогічно кластеризувати клієнтів або магазини.\n"
      ]
    },
    {
      "cell_type": "code",
      "metadata": {},
      "execution_count": null,
      "outputs": [],
      "source": [
        "# Агрегація по товару\n",
        "group_key = 'product_id' if 'product_id' in df_fe.columns else None\n",
        "if group_key is None:\n",
        "    raise ValueError('Немає колонки product_id. Адаптуй під свої дані або вимкни цей блок.')\n",
        "\n",
        "agg = df_fe.groupby(group_key).agg(\n",
        "    sales_sum=('sales', 'sum'),\n",
        "    qty_sum=('quantity', 'sum'),\n",
        "    price_mean=('price', 'mean'),\n",
        "    rows=('sales', 'size')\n",
        ").reset_index()\n",
        "\n",
        "agg.head(10)\n"
      ]
    },
    {
      "cell_type": "code",
      "metadata": {},
      "execution_count": null,
      "outputs": [],
      "source": [
        "# Масштабування\n",
        "feat_cols = ['sales_sum', 'qty_sum', 'price_mean', 'rows']\n",
        "X = agg[feat_cols].fillna(0.0).to_numpy()\n",
        "\n",
        "scaler = StandardScaler()\n",
        "X_scaled = scaler.fit_transform(X)\n",
        "\n",
        "X_scaled[:3]\n"
      ]
    },
    {
      "cell_type": "code",
      "metadata": {},
      "execution_count": null,
      "outputs": [],
      "source": [
        "# Підбір k через silhouette\n",
        "scores = []\n",
        "k_range = range(2, 9)\n",
        "for k in k_range:\n",
        "    km = KMeans(n_clusters=k, random_state=42, n_init=10)\n",
        "    labels = km.fit_predict(X_scaled)\n",
        "    scores.append(silhouette_score(X_scaled, labels))\n",
        "\n",
        "plt.figure(figsize=(6, 3.5))\n",
        "plt.plot(list(k_range), scores, marker='o')\n",
        "plt.title('Silhouette Score для вибору кількості кластерів')\n",
        "plt.xlabel('k')\n",
        "plt.ylabel('silhouette')\n",
        "plt.grid(True, alpha=0.3)\n",
        "plt.show()\n"
      ]
    },
    {
      "cell_type": "code",
      "metadata": {},
      "execution_count": null,
      "outputs": [],
      "source": [
        "# Фінальна кластеризація\n",
        "k_final = 3\n",
        "kmeans = KMeans(n_clusters=k_final, random_state=42, n_init=10)\n",
        "agg['cluster'] = kmeans.fit_predict(X_scaled)\n",
        "agg.head(10)\n"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {},
      "source": [
        "## 9. Прогнозування продажів (ML)\n",
        "\n",
        "Ціль: `sales`.\n",
        "Базові ознаки (приклад): `price`, `quantity`, `month`, `dow`.\n",
        "\n",
        "У реальній роботі ознаки можна розширити:\n",
        "- лаги продажів;\n",
        "- ковзні середні;\n",
        "- категоріальні ознаки (магазин/категорія) через one-hot;\n",
        "- техніки боротьби з викидами.\n"
      ]
    },
    {
      "cell_type": "code",
      "metadata": {},
      "execution_count": null,
      "outputs": [],
      "source": [
        "required_cols = ['sales', 'price', 'quantity']\n",
        "for c in required_cols:\n",
        "    if c not in df_fe.columns:\n",
        "        raise ValueError(f'Немає колонки {c}. Адаптуй під свої дані.')\n",
        "\n",
        "feature_cols = ['price', 'quantity']\n",
        "if 'month' in df_fe.columns:\n",
        "    feature_cols.append('month')\n",
        "if 'dow' in df_fe.columns:\n",
        "    feature_cols.append('dow')\n",
        "\n",
        "X = df_fe[feature_cols].copy()\n",
        "y = df_fe['sales'].copy()\n",
        "\n",
        "X_train, X_test, y_train, y_test = train_test_split(\n",
        "    X, y, test_size=0.2, random_state=42\n",
        ")\n",
        "\n",
        "X_train.shape, X_test.shape\n"
      ]
    },
    {
      "cell_type": "code",
      "metadata": {},
      "execution_count": null,
      "outputs": [],
      "source": [
        "# Baseline: Лінійна регресія\n",
        "lr = LinearRegression()\n",
        "lr.fit(X_train, y_train)\n",
        "pred_lr = lr.predict(X_test)\n",
        "\n",
        "mae_lr = mean_absolute_error(y_test, pred_lr)\n",
        "rmse_lr = mean_squared_error(y_test, pred_lr, squared=False)\n",
        "\n",
        "print('Linear Regression')\n",
        "print('MAE :', round(mae_lr, 4))\n",
        "print('RMSE:', round(rmse_lr, 4))\n"
      ]
    },
    {
      "cell_type": "code",
      "metadata": {},
      "execution_count": null,
      "outputs": [],
      "source": [
        "# Модель: Random Forest\n",
        "rf = RandomForestRegressor(\n",
        "    n_estimators=250,\n",
        "    random_state=42,\n",
        "    n_jobs=-1\n",
        ")\n",
        "rf.fit(X_train, y_train)\n",
        "pred_rf = rf.predict(X_test)\n",
        "\n",
        "mae_rf = mean_absolute_error(y_test, pred_rf)\n",
        "rmse_rf = mean_squared_error(y_test, pred_rf, squared=False)\n",
        "\n",
        "print('Random Forest')\n",
        "print('MAE :', round(mae_rf, 4))\n",
        "print('RMSE:', round(rmse_rf, 4))\n"
      ]
    },
    {
      "cell_type": "code",
      "metadata": {},
      "execution_count": null,
      "outputs": [],
      "source": [
        "# Порівняння моделей\n",
        "results = pd.DataFrame({\n",
        "    'Модель': ['Linear Regression', 'Random Forest'],\n",
        "    'MAE': [mae_lr, mae_rf],\n",
        "    'RMSE': [rmse_lr, rmse_rf]\n",
        "})\n",
        "\n",
        "results.sort_values('RMSE')\n"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {},
      "source": [
        "## 10. Висновки\n",
        "\n",
        "У роботі реалізовано відтворюваний пайплайн обробки неструктурованих Excel-звітів:\n",
        "- завантаження та уніфікація колонок;\n",
        "- очистка й підготовка;\n",
        "- EDA (зокрема boxplot для оцінки викидів);\n",
        "- кластеризація товарів за агрегованими характеристиками;\n",
        "- побудова моделей машинного навчання для прогнозування продажів;\n",
        "- порівняння якості моделей за MAE/RMSE.\n",
        "\n",
        "### Напрями розвитку\n",
        "- часові ряди (lags/rolling, Prophet, LSTM);\n",
        "- автоматичне розпізнавання структури Excel (пошук хедерів, пропуск “сміттєвих” рядків);\n",
        "- побудова API/веб-інтерфейсу для завантаження звітів та отримання прогнозів.\n"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {},
      "source": [
        "## Додаток: контроль відтворюваності\n",
        "\n",
        "Перед захистом бажано:\n",
        "1) `Runtime → Restart and run all` (у Colab)\n",
        "2) Переконатися, що шлях до Excel правильний\n",
        "3) Зафіксувати фінальну версію ноутбука (GitHub commit / копія у Drive)\n"
      ]
    }
  ]
}
